# Lane Finder (part 2)

Now that we have managed to get a somewhat reliable baseline that crosses the peak of the true lane line, we are ready to move on and construct the actual lane line.

Note that this is somewhat different than trying to project first, but I am following my intuition.

In [3]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import itertools
import numbers
import numpy as np
import cv2

def binary_array_cleanup(array):
    if type(array) != np.ndarray:
        raise TypeError
        
    select_high = array > (array.dtype.type(~0) >> 1)
    select_low = ~select_high
    
    array[select_high] = array.dtype.type(~0)
    array[select_low] = array.dtype.type(0)
    
    return array

def binary_array_peaks(vector, min_width = 10, max_width = 75):
        if type(vector) != np.ndarray:
            raise TypeError()
        elif vector.ndim > 1:
            raise ValueError()
        
        if np.count_nonzero(vector) > 0.75 * vector.size:
            active_value = False
        else:
            active_value = True
        
        index = 0
        for value, group in itertools.groupby(vector):
            length = len(list(group))
            if active_value == bool(value) and length >= min_width and length <= max_width:
                yield int((index + length / 2) - (vector.size / 2))
                
            index += length
            
def binary_array_peaks2(vector, min_width = 10, max_width = 75):
        if type(vector) != np.ndarray:
            raise TypeError()
        elif vector.ndim > 1:
            raise ValueError()
        
        if np.count_nonzero(vector) > 0.75 * vector.size:
            active_value = False
        else:
            active_value = True
        
        index = 0
        for value, group in itertools.groupby(vector):
            length = len(list(group))
            if active_value == bool(value) and length >= min_width and length <= max_width:
                yield int((index + length / 2) - (vector.size / 2)), length
                
            index += length

# use example:
#
# process_image = lane_manager( \
#    lane_baseline(...leftlane...), \
#    lane_baseline(..rightlane..), \
#    callable(img, leftlane, rightlane),
#    draw_borderline=True)
#
# In other words, create an instance of this class while feeding two lane
# baseline class instances one for the left and one for the right lanes. 
# You also have to pass a callable object or a function that accepts three
# operands, one for the GB binary video frame while the other two are for
# the lane borderline objects. This will allow you to build upon my baseline
# finding algorithm.
class lane_manager:
    def __init__(self, leftlane, rightlane, user_callable, draw_borderline=True):
        if not isinstance(leftlane, lane_baseline) or \
           not isinstance(rightlane, lane_baseline) or \
           not callable(user_callable):
            raise TypeError()
        self.leftlane  = leftlane
        self.rightlane = rightlane
        self.lanewidth = 720
        self.lanewidth_reliability = 0.5
        self.user_callable = user_callable
        self.draw_borderline = draw_borderline 
    def __call__(self, img):
        if bool(self.leftlane) and bool(self.rightlane):
            rightlane_center, na = self.rightlane.newx(img)
            leftlane_center , na = self.leftlane.newx(img)
            rightlane_center += self.rightlane.neww(img) // 2
            leftlane_center  += self.leftlane.neww(img)  // 2
            measurement = abs(rightlane_center - leftlane_center)
            self.lanewidth += int((measurement - self.lanewidth) / 12.5)
            self.lanewidth_reliability = 1.0 - (abs(self.lanewidth - measurement) / self.lanewidth)
        else:
            self.lanewidth_reliability -= (0.15 / 25)
            
        if self.lanewidth_reliability > 0.75:
            if self.rightlane < self.leftlane:
                self.rightlane.set_center(self.leftlane.get_center() + self.lanewidth)
            if self.leftlane < self.rightlane:
                self.leftlane.set_center(self.rightlane.get_center() - self.lanewidth)
                    
        x1, y1, w1, text1 = self.leftlane(img)
        x2, y2, w2, text2 = self.rightlane(img)
        
        if self.draw_borderline:
            cv2.line(img, (x1, y1), (x1 + w1, y1), (255, 0, 0), 4)
            cv2.line(img, (x2, y2), (x2 + w2, y2), (255, 255, 0), 4)
        
        return self.user_callable(img, self.leftlane, self.rightlane)
        
class lane_baseline:
    def __init__(self, x, y, min_width, max_width):
        if type(x) != int or type(y) != int or \
           type(min_width) != int or type(max_width) != int:
            raise TypeError()
        self.x  = x
        self.x0 = x
        self.y  = y
        self.w  = min_width
        self.w_min = min_width
        self.w_max = max_width
        self.idle_counter = 0
        self.previous_img = None
        self.blue_reliability  = 1.0
        self.green_reliability = 1.0
        self.blue_peaks  = []
        self.green_peaks = []
        self.decision_index = 0
        self.external_setting = False
        self.outside_frame = False
    def __bool__(self):
        return (len(self.green_peaks) > 0 and self.green_reliability > 0.75) or \
            (len(self.blue_peaks) > 0 and self.blue_reliability > 0.80)
    def __lt__(self, other):
        if self.decision_index < other.decision_index:
            return True
        if self.decision_index == other.decision_index:
            if 2 == self.decision_index:
                return self.green_reliability < self.green_reliability
            elif 1 == self.decision_index:
                return self.blue_reliability < self.blue_reliability
    def get_center(self):
        return int(self.x + (self.w/2))
    def set_center(self, x_center):
        self.x_center = int(x_center)
        self.external_setting = True
    def neww(self, img):
        neww = self.w
        neww -= int((self.w - self.w_min) / 6.25)
        neww += 1 << (self.idle_counter >> 1)
        neww = max(self.w_min, neww)
        neww = min(self.w_max, neww)
        return neww
    def newreliability(self, img):
        gr, br = self.green_reliability, self.blue_reliability
        w = self.neww(self)
        if self.previous_img is not None:
            diff = binary_array_cleanup(cv2.absdiff(img, self.previous_img) \
                [self.y-10:self.y+10, max(0,self.x-50):min(img.shape[1],self.x+w),:])
            if diff.size > 0:
                gr *= 1.0 - (np.count_nonzero(diff[:,:,1]) / diff[:,:,1].size)
                br *= 1.0 - (np.count_nonzero(diff[:,:,2]) / diff[:,:,2].size)    
        gr += (1.0 - gr) / 6.25
        br += (1.0 - br) / 6.25
        return gr, br
    def newx(self, img):
        w = self.neww(self)
        baseline_green = img[self.y, max(0,self.x):min(img.shape[1],self.x+w), 1]
        baseline_blue = img[self.y, max(0,self.x):min(img.shape[1],self.x+w), 2]
        gr, br = self.newreliability(img)
        gp, bp = None, None
        if len(baseline_green) > 0 and len(baseline_blue) > 0:
            gp = list(binary_array_peaks(baseline_green))
            bp  = list(binary_array_peaks(baseline_blue, 2))
            if not self.external_setting:
                if len(gp) > 0 and gr > 0.75:
                    x = self.x + int(np.median(gp))
                elif len(bp) > 0 and br > 0.80:
                    x = self.x + int(np.median(bp))
                else:
                    x = self.x + int((self.x0 - self.x) / 12.5)
            else:
                x = self.x_center - (w//2)
        return x, (baseline_green, baseline_blue, gp, bp)
    def __call__(self, img, change_state=True):
        # COMPUTE STATE
        w = self.neww(img)
        gr, br = self.newreliability(img)
        x, (baseline_green, baseline_blue, gp, bp) = self.newx(img)
        text = ""
        # CHANGE STATE
        if (change_state): # no text will be provided
            self.previous_img = np.copy(img)
            self.w = w
            self.green_reliability = gr
            self.blue_reliability = br
            self.x = x
            if len(baseline_green) > 0 and len(baseline_blue) > 0:
                self.outside_frame = False
                self.green_peaks = gp
                self.blue_peaks  = bp
                if not self.external_setting:
                    if len(gp) > 0 and gr > 0.75:
                        self.idle_counter = 0
                        self.decision_index = 2
                        text += "G"
                    elif len(bp) > 0 and br > 0.80:
                        self.idle_counter = 0
                        self.decision_index = 1
                        text += "B"
                    else:
                        self.idle_counter += 1
                        self.decision_index = 0
                        text += "-"
                else:
                    self.idle_counter = 0
                    self.decision_index = -1
                    self.external_setting = False
                    text += "X"
            else:
                self.idle_counter += 1
                self.decision_index = 0
                self.outside_frame = True
                self.green_peaks = []
                self.blue_peaks = []
        return int(x), int(self.y), int(w), text

## Vanishing Point

In [ ]:
def process_image(img, leftlane, rightlane):
    img = binary_array_cleanup(img)
    img_gray = img[:,:,1]
    lines = cv2.HoughLines(img_gray, )
    return img

input_video = VideoFileClip("challenge_video_binaryGB_undistorted_cut.mp4")
output_video = input_video.fl_image(lane_manager( \
    lane_baseline(205, 670, 190, 300), \
    lane_baseline(920, 670, 190, 300), \
    process_image))

%time output_video.write_videofile('challenge_video_binaryGB_undistorted_lanefinder.mp4', audio=False)

In [8]:
def process_image(img, leftlane, rightlane):
    
    return img

input_video = VideoFileClip("challenge_video_binaryGB_undistorted_cut.mp4")
output_video = input_video.fl_image(lane_manager( \
    lane_baseline(205, 670, 190, 300), \
    lane_baseline(920, 670, 190, 300), \
    process_image))

%time output_video.write_videofile('challenge_video_binaryGB_undistorted_lanefinder.mp4', audio=False)

[(-29, 30)]


Exception: 